# Australian COVID Vaccination eligibility and up-to-date (circa mid-2022)
This example explores the COVID vaccination eligibility and up-to-date regime in place in the middle of 2022.

The relation (table) containing the rules has the following columns:
```
 covid_vaccinations(age,
                    recommended_doses,
                    booster_doses,
                    vaccine_doses,
                    immunocompromised_disability,
                    months_since_last_dose_at_least,
                    covid_vaccination_eligibility,
                    covid_vaccination_up_to_date)
```

Here are some examples:

If a person's last COVID vaccination was 5 months ago, are 29, and have had four COVID Vaccinations.  Are they up-to-date?

In [1]:
%reload_ext jetisu.query_idr_magic

In [2]:
%%jetisu_query
select distinct covid_vaccination_up_to_date, covid_vaccination_eligibility from covid_vaccinations where months_since_last_dose_at_least = 5 and age=29 and vaccine_doses = 4;

|covid_vaccination_up_to_date|covid_vaccination_eligibility|
|----|----|
|True|False|

However, if the person's age was 70 then the answer is mixed, the query shows that the answer is maybe yes, maybe no.

In [3]:
%%jetisu_query
select distinct covid_vaccination_up_to_date, covid_vaccination_eligibility from covid_vaccinations where months_since_last_dose_at_least = 5 and age=70 and vaccine_doses = 4;

|covid_vaccination_up_to_date|covid_vaccination_eligibility|
|----|----|
|False|True|
|True|False|

If we show all the columns in the table, we can see which factor makes the difference the True/ False outcomes.  The following table shows that it is ```immunocompromised_disability```, which lifts the ```recommended_doses``` to 3, which along with the ```booster_doses``` gives a total of 5 being recommended.

In [4]:
%%jetisu_query
select distinct * from covid_vaccinations where months_since_last_dose_at_least = 5 and age=70 and vaccine_doses = 4;

|age|recommended_doses|booster_doses|vaccine_doses|immunocompromised_disability|months_since_last_dose_at_least|covid_vaccination_eligibility|covid_vaccination_up_to_date|
|----|----|----|----|----|----|----|----|
|70.0|2|2|4.0|False|5.0|False|True|
|70.0|3|2|4.0|True|5.0|True|False|

So, now we can see that the 70 year old is not up-to-date if immunocompromised:

In [5]:
%%jetisu_query
select distinct covid_vaccination_up_to_date, covid_vaccination_eligibility from covid_vaccinations where immunocompromised_disability and months_since_last_dose_at_least = 5 and age=70 and vaccine_doses = 4;

|covid_vaccination_up_to_date|covid_vaccination_eligibility|
|----|----|
|False|True|

And if the 70 year old had had their last dose, just 2 months ago, they would get a different response. The person is not up to date, and neither are they eligible!

In [6]:
%%jetisu_query
select distinct covid_vaccination_up_to_date, covid_vaccination_eligibility from covid_vaccinations where immunocompromised_disability and months_since_last_dose_at_least = 2 and age=70 and vaccine_doses = 4;

|covid_vaccination_up_to_date|covid_vaccination_eligibility|
|----|----|
|False|False|

## Definition in MiniZinc

In [7]:
%%jetisu_show
covid_vaccinations

```

predicate covid_vaccinations(var int: age,
                var int: recommended_doses,
                var int: booster_doses,
                var int: vaccine_doses,
                var bool: immunocompromised_disability,
                var int: months_since_last_dose_at_least,
                var bool: covid_vaccination_eligibility,
                var bool: covid_vaccination_up_to_date
                ) =
let {
  par int: highest_number_of_months_since_last_dose_that_we_care_about = 6;
  par int: highest_number_of_vaccine_doses_that_we_care_about = 5;

  constraint age >= 0 /\ age <=115;
  constraint vaccine_doses >= 0 /\ vaccine_doses <= highest_number_of_vaccine_doses_that_we_care_about;
  constraint vaccine_doses = 0 -> months_since_last_dose_at_least = highest_number_of_months_since_last_dose_that_we_care_about;
  constraint covid_vaccination_up_to_date -> not covid_vaccination_eligibility;
  constraint months_since_last_dose_at_least >= 0 /\ months_since_last_dose_at_least <=highest_number_of_months_since_last_dose_that_we_care_about;
  constraint recommended_doses = if     age <   5              then 0
                                 elseif age >=  5 /\ age <= 11 then 2 + if immunocompromised_disability then 1 else 0 endif
                                 elseif age >= 12 /\ age <= 15 then 2 + if immunocompromised_disability then 1 else 0 endif
                                 elseif age >= 16 /\ age <= 29 then 2 + if immunocompromised_disability then 1 else 0 endif
                                 elseif age >= 30 /\ age <= 49 then 2 + if immunocompromised_disability then 1 else 0 endif
                                 else 2 + if immunocompromised_disability then 1 else 0 endif
                                 endif;
constraint booster_doses = if     age <   5              then 0
                                 elseif age >=  5 /\ age <= 11 then 0
                                 elseif age >= 12 /\ age <= 15 then 0
                                 elseif age >= 16 /\ age <= 29 then 1
                                 elseif age >= 30 /\ age <= 49 then 1
                                 else 2
                                 endif;

constraint covid_vaccination_eligibility = (months_since_last_dose_at_least >= 3 /\ (recommended_doses+booster_doses > vaccine_doses));
constraint covid_vaccination_up_to_date = (recommended_doses + booster_doses <= vaccine_doses) /\ ((recommended_doses + booster_doses > vaccine_doses) -> (months_since_last_dose_at_least <= 6));
} in true;
```

## Test Cases
In addition to these examples, the ```covid_vaccinations``` intensionally defined relation passes an additional 66 tests in ```test_covid_vaccinations.py```

## Next step
You can edit and re-run this example workbook by following the instructions here:
[How to run the example notebooks](https://github.com/DavidPratten/jetisu/blob/main/docs/run_notebooks.md)